In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import statsmodels.api as sm

/Users/michael/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

In [3]:
%matplotlib inline
pd.options.display.max_columns = 500
plt.style.use('ggplot')

In [4]:
np.random.seed(12345)

# Modeling

## Train Test split

We will set aside 20% of the data for testing, maintaining the distribution of yes/no in the vehicle column across the training and testing sets.

In [5]:
data = pd.read_csv('ebike_survey_responses_processed.csv')

In [6]:
data.shape

(2227, 86)

In [7]:
data.head()

,health,education,income,daily_travel_distance,commute_time,vehicles,speed_limit_aware,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,a0,a1,a2,a3,a4,a5,a6,a7,c0,c1,c2,c3,c4,c5,c6,speeding_action,m0,m1,m2,m3,m4,b_motor,b_do_not_use,b_bike_other,eb0,eb1,eb2,eb3,eb4,eb5,si0,si1,si2,si3,mo0,mo1,mo2,mo3,age_17 or younger,age_18 to 34,age_35 to 49,age_50 to 64,age_65 years or more,sex_Female,sex_Male,sex_Other,employment_disabled,employment_full_time,employment_home,employment_other,employment_part_time,employment_retired,employment_self_employed,employment_student,employment_unemployed,district_Central Toronto York or East York,district_Etobicoke,district_Mississauga,district_North York,district_Other District,district_Scarborough,transportation_bicycle,transportation_pedal,transportation_personal_mobility,transportation_private_motor,transportation_scooter,transportation_transit,transportation_walking
0,3,4,6.0,1,2,1,0,0,1,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1,5,3,3.0,3,4,1,0,0,1,1,0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,1,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
2,3,3,3.0,3,2,1,0,0,0,0,1,1,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
3,3,3,5.0,1,2,1,0,0,1,1,1,1,0,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
4,4,2,3.0,2,2,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0


In [8]:
y = data['vehicles']
X = data.drop(['vehicles'], axis=1)

In [9]:
def independent_columns(A, tol = 1e-05):
    """
    Return an array composed of independent columns of A
    Adapted from https://stackoverflow.com/questions/13312498/how-to-find-degenerate-rows-columns-in-a-covariance-matrix/13313828#13313828

    Note the answer may not be unique; this function returns one of many
    possible answers.

    http://stackoverflow.com/q/13312498/190597 (user1812712)
    http://math.stackexchange.com/a/199132/1140 (Gerry Myerson)
    http://mail.scipy.org/pipermail/numpy-discussion/2008-November/038705.html
        (Anne Archibald)

    >>> A = np.array([(2,4,1,3),(-1,-2,1,0),(0,0,2,2),(3,6,2,5)])
    >>> independent_columns(A)
    np.array([[1, 4],
              [2, 5],
              [3, 6]])
    """
    Q, R = np.linalg.qr(A)
    independent = np.where(np.abs(R.diagonal()) > tol)[0]
    return A.iloc[:, independent]

X = independent_columns(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

In [11]:
# We observed that scaling data actually worsened performance. Hence we disable scaling.
# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

## Baseline Model

For the baseline model, we simply predict the majority class.

In [12]:
def report_scores(scores):
    for k, v in scores.items():
        if k.startswith('train') or k.startswith('test'):
            print(k, f'{v.mean():.3f} (+/- {v.std() * 2:.3f})')

In [13]:
scoring = ['accuracy', 'f1', 'roc_auc']

baseline_model = DummyClassifier(strategy='most_frequent')
scores = cross_validate(baseline_model, X_train, y_train, scoring=scoring, cv=5, return_train_score=True)
report_scores(scores)

test_accuracy 0.765 (+/- 0.002)
train_accuracy 0.765 (+/- 0.000)
test_f1 0.867 (+/- 0.001)
train_f1 0.867 (+/- 0.000)
test_roc_auc 0.500 (+/- 0.000)
train_roc_auc 0.500 (+/- 0.000)


## Logistic Regression

Start with a simple logistic regression model, which doesn't have hyperparameters we need to tune.

In [14]:
logistic_model = LogisticRegression()
scores = cross_validate(logistic_model, X_train, y_train, scoring=scoring, cv=5, return_train_score=True)
report_scores(scores)

test_accuracy 0.805 (+/- 0.021)
train_accuracy 0.824 (+/- 0.005)
test_f1 0.878 (+/- 0.012)
train_f1 0.890 (+/- 0.003)
test_roc_auc 0.823 (+/- 0.044)
train_roc_auc 0.872 (+/- 0.011)


In [15]:
sm_logit_model = sm.Logit(y_train, X_train)
sm_result = sm_logit_model.fit(method='bfgs')
print(sm_result.summary())

         Current function value: 0.376705
         Iterations: 35
         Function evaluations: 37
         Gradient evaluations: 37
                           Logit Regression Results                           
Dep. Variable:               vehicles   No. Observations:                 1781
Model:                          Logit   Df Residuals:                     1701
Method:                           MLE   Df Model:                           79
Date:                Sat, 12 May 2018   Pseudo R-squ.:                  0.3095
Time:                        11:45:45   Log-Likelihood:                -670.91
converged:                      False   LL-Null:                       -971.63
                                        LLR p-value:                 2.258e-81
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
health      

/Users/michael/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [16]:
p_values = list(zip(X.columns, sm_result.pvalues))
p_values.sort(key=lambda k: k[1])
p_values

[('income', 1.4329802224819224e-16),
 ('transportation_private_motor', 4.0597339173149075e-10),
 ('c2', 0.0026123504960698183),
 ('a6', 0.004643488834474009),
 ('s7', 0.0064340199799708744),
 ('daily_travel_distance', 0.008464917839580881),
 ('eb2', 0.019658157458150853),
 ('transportation_bicycle', 0.05698705236963584),
 ('s6', 0.06683642132367924),
 ('education', 0.07106277885954872),
 ('eb4', 0.08270975440669313),
 ('s1', 0.10586842082114273),
 ('s9', 0.12102289652226957),
 ('transportation_pedal', 0.12680200380720083),
 ('commute_time', 0.14089000333187265),
 ('district_Central Toronto York or East York', 0.1453300524974454),
 ('eb3', 0.15600067513362445),
 ('a5', 0.15852725691449965),
 ('transportation_scooter', 0.18572433726993276),
 ('mo1', 0.19317977296099154),
 ('m1', 0.19442982889241256),
 ('speed_limit_aware', 0.19982609612013902),
 ('c1', 0.20001036093521252),
 ('si2', 0.21126166987354578),
 ('a1', 0.21880088913002338),
 ('age_18 to 34', 0.2602645356977684),
 ('s5', 0.27444

## Support Vector Machine

### RBF Kernel (default)

In [17]:
svc = SVC()
scores = cross_validate(svc, X_train, y_train, scoring=scoring, cv=5, return_train_score=True)
report_scores(scores)

test_accuracy 0.794 (+/- 0.012)
train_accuracy 0.814 (+/- 0.012)
test_f1 0.877 (+/- 0.008)
train_f1 0.888 (+/- 0.006)
test_roc_auc 0.814 (+/- 0.037)
train_roc_auc 0.872 (+/- 0.005)


### Polynomial Kernel

In [18]:
svc_poly = SVC(kernel='poly', degree=3)
scores = cross_validate(svc_poly, X_train, y_train, scoring=scoring, cv=5, return_train_score=True)
report_scores(scores)

test_accuracy 0.792 (+/- 0.005)
train_accuracy 0.810 (+/- 0.010)
test_f1 0.878 (+/- 0.002)
train_f1 0.888 (+/- 0.004)
test_roc_auc 0.805 (+/- 0.043)
train_roc_auc 0.881 (+/- 0.008)


## Random Forest

In [19]:
rf_classifier = RandomForestClassifier()
scores = cross_validate(rf_classifier, X_train, y_train, scoring=scoring, cv=5, return_train_score=True)
report_scores(scores)

test_accuracy 0.775 (+/- 0.028)
train_accuracy 0.993 (+/- 0.002)
test_f1 0.857 (+/- 0.018)
train_f1 0.995 (+/- 0.001)
test_roc_auc 0.765 (+/- 0.019)
train_roc_auc 1.000 (+/- 0.000)


In [20]:
rf_classifier.fit(X_train, y_train)
feature_importances = list(zip(X.columns, rf_classifier.feature_importances_))
feature_importances.sort(reverse=True, key=lambda p: p[1])
feature_importances

[('income', 0.071259982742194194),
 ('transportation_bicycle', 0.038927115644431964),
 ('health', 0.035437205417421215),
 ('commute_time', 0.034223788920281842),
 ('daily_travel_distance', 0.033930320701548264),
 ('transportation_private_motor', 0.032777622125914861),
 ('district_Central Toronto York or East York', 0.032766421245970928),
 ('age_18 to 34', 0.030378541351457476),
 ('education', 0.02367570012184831),
 ('si3', 0.019534136758630749),
 ('s7', 0.017602317306399242),
 ('si0', 0.017503245112508916),
 ('s5', 0.017172386225363465),
 ('s9', 0.017072031845459169),
 ('s8', 0.016664332733851352),
 ('s3', 0.01655900744160814),
 ('speed_limit_aware', 0.016336062131135975),
 ('employment_full_time', 0.016122717767759715),
 ('si2', 0.016114511588266857),
 ('eb1', 0.01598662218138076),
 ('s1', 0.015492454252675727),
 ('m0', 0.015416250872780934),
 ('a0', 0.015237954283693378),
 ('s2', 0.015103184407352272),
 ('a1', 0.014666119481065113),
 ('sex_Male', 0.013969884155173931),
 ('si1', 0.013

## Neural Network

In [21]:
mlp_classifier = MLPClassifier((50, ), solver='sgd', max_iter=300)
scores = cross_validate(mlp_classifier, X_train, y_train, scoring=scoring, cv=5, return_train_score=True)
report_scores(scores)

test_accuracy 0.788 (+/- 0.013)
train_accuracy 0.813 (+/- 0.010)
test_f1 0.871 (+/- 0.011)
train_f1 0.885 (+/- 0.005)
test_roc_auc 0.815 (+/- 0.026)
train_roc_auc 0.839 (+/- 0.011)


## Gradient Boosting

In [22]:
gb_classifier = GradientBoostingClassifier()
scores = cross_validate(gb_classifier, X_train, y_train, scoring=scoring, cv=5, return_train_score=True)
report_scores(scores)

test_accuracy 0.789 (+/- 0.012)
train_accuracy 0.865 (+/- 0.009)
test_f1 0.868 (+/- 0.010)
train_f1 0.916 (+/- 0.006)
test_roc_auc 0.820 (+/- 0.037)
train_roc_auc 0.936 (+/- 0.003)


## Randomized Grid Search

In [23]:
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [2, 3, 4, 5]
}
gs = GridSearchCV(GradientBoostingClassifier(random_state=1234),
                          param_grid=param_grid,
                          scoring=scoring, cv=5, refit='roc_auc')
gs.fit(X_train, y_train)
results = gs.cv_results_

In [24]:
results_list = list(zip(results['params'], results['rank_test_roc_auc'], results['mean_test_roc_auc'], results['rank_test_f1'], results['mean_test_f1']))

In [25]:
sorted(results_list, key=lambda k: k[1])

[({'max_depth': 2, 'n_estimators': 100},
  1,
  0.82501216229435903,
  1,
  0.87408070550690664),
 ({'max_depth': 2, 'n_estimators': 50},
  2,
  0.82472310892834,
  4,
  0.87222866224350137),
 ({'max_depth': 2, 'n_estimators': 200},
  3,
  0.82249221072527134,
  3,
  0.87248471575815101),
 ({'max_depth': 3, 'n_estimators': 50},
  4,
  0.81984882593089614,
  5,
  0.87032642761774148),
 ({'max_depth': 3, 'n_estimators': 100},
  5,
  0.81952822887775989,
  8,
  0.86800688628182376),
 ({'max_depth': 2, 'n_estimators': 300},
  6,
  0.81752521244393184,
  2,
  0.87283685960337842),
 ({'max_depth': 4, 'n_estimators': 50},
  7,
  0.81362044903562758,
  6,
  0.86895922894628352),
 ({'max_depth': 3, 'n_estimators': 200},
  8,
  0.81293299731706392,
  9,
  0.86522187612473767),
 ({'max_depth': 3, 'n_estimators': 300},
  9,
  0.80925788150207167,
  13,
  0.86151893081337927),
 ({'max_depth': 5, 'n_estimators': 50},
  10,
  0.80780949680771164,
  10,
  0.86491039084363153),
 ({'max_depth': 4, 'n_es

## Final Model

In [26]:
final_model = GradientBoostingClassifier(n_estimators=100, max_depth=2)
final_model.fit(X_train, y_train)
y_test_predict = final_model.predict_proba(X_test)

test_roc_auc = roc_auc_score(y_test, y_test_predict[:, 1])
test_f1 = f1_score(y_test, y_test_predict[:, 1] >= 0.5)
test_acc = accuracy_score(y_test, y_test_predict[:, 1] >= 0.5)

print(f'Test ROC AUC: {test_roc_auc:.3f}')
print(f'Test F1: {test_f1:.3f}')
print(f'Test Accuracy: {test_acc * 100:.3f}%')

Test ROC AUC: 0.895
Test F1: 0.898
Test Accuracy: 83.408%


Top 10 important features ranked by decreasing order of importance:

In [27]:
feature_importances = list(zip(X.columns, final_model.feature_importances_))
feature_importances.sort(reverse=True, key=lambda p: p[1])
feature_importances[:10]

[('income', 0.14570680896504565),
 ('transportation_private_motor', 0.099920837161514023),
 ('s9', 0.050663763243510601),
 ('transportation_bicycle', 0.045270803680559621),
 ('c2', 0.041939525088299352),
 ('transportation_pedal', 0.040714473708275717),
 ('daily_travel_distance', 0.036378843333114197),
 ('age_18 to 34', 0.035872911410122027),
 ('transportation_scooter', 0.031048465340632447),
 ('district_Central Toronto York or East York', 0.03101523689475846)]